In [ ]:
import numpy as np
import pymc as pm
import pandas as pd
import aesara.tensor as at
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from scipy import stats
import logging
import extract_correct_csv
from deepemogp import feature_extractor
from deepemogp.signal import physio as physio
from deepemogp import datasets as datasets
from deepemogp.signal import behavior as behavior

In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
all_valid_subject = extract_correct_csv.extract_only_valid_subject()

num_trials_to_remove = 0
extraction_method = 'wavelet'
latent_space = 3
use_sampling = 'n'
if use_sampling == 'y':
    use_sampling = True
else:
    use_sampling = False

In [ ]:
prova_3_subj = [2, 4, 10]

In [ ]:
global_e_labels = []
global_subject = []
for i in prova_3_subj:
    subj_ = extract_correct_csv.read_correct_subject_csv(i)
    csv_ = '/Users/marcoghezzi/PycharmProjects/pythonProject/osfstorage-archive/behavior/LookAtMe_0' + subj_ + '.csv'
    #csv_ = '/data/notebook_files/behavior/LookAtMe_0'+str(subj_)+'.csv'
    #csv_ = '/home/paolo/matteo/matteo/unimi/tesi_master/code/osfstorage-archive/behavior/LookAtMe_0'+str(subj_)+'.csv'
    global_data = pd.read_csv(csv_, sep='\t')
    y = np.array(list([int(d>2) for d in global_data['rating']]))
    e_labels = y[:,np.newaxis]  # rating > 2
    global_e_labels = global_e_labels + e_labels.tolist()
    subject = np.array(list([s for s in global_data['subject']]))[:, np.newaxis]
    global_subject = global_subject + subject.tolist()
#e_labels = e_labels[num_trials_to_remove:]
'''N_e = e_labels.shape[0]
D_e = e_labels.shape[1]'''
global_e_labels = np.array(global_e_labels)
global_subject = np.array(global_subject)

N_e = global_e_labels.shape[0]
D_e = global_e_labels.shape[1]
print(N_e, D_e)

N_sub = global_subject.shape[0]
D_sub = global_subject.shape[1]
print(N_sub, D_sub)

In [ ]:
show = False
# definition of the feature extractors to be used later
f1 = feature_extractor.FE('wavelet', window=(2, 1))
f2 = feature_extractor.FE('mean', window=(1,0))

        # definition of the physiological signals to be extracted
if extraction_method == 'wavelet':
    eda_ = physio.EDA(f1)
    hr_ = physio.HR(f1)
else:
    eda_ = physio.EDA(f2)
    hr_ = physio.HR(f2)
pupil_ = behavior.PUPIL(f2)

elenco_subj = ([str(d) for d in prova_3_subj])
# extraction of the desired data from the dataset
d = datasets.FEAR(signals={hr_,pupil_,eda_}, subjects=set(elenco_subj))


In [ ]:
for s in d.signals:
    # preprocess ...
    if s.name =='EDA':
        s.preprocess(show=show,new_fps=500)
        s.feature_ext.extract_feat(s,show=show)
    else:
        if s.name == 'HR':
            list_hr_test = s.raw[0]['data']
            s.preprocess(show=show, useneurokit=True)
            s.feature_ext.extract_feat(s,show=show)

        else:
            s.feature_ext.extract_feat_without_preprocess(s, show=show)

            #add feature extraction for eda before preprocessing

            # ... and extract features from each signal type


for sig in d.signals:
    if sig.name=='EDA':
        eda_data = sig.features
    if sig.name=='HR':
        hr_data = sig.features
    if sig.name=='PUPIL':
        pupil_data = sig.features

In [ ]:
TRIAL = 160*len(prova_3_subj)
hr =np.array(hr_data)
hr = hr.reshape((TRIAL, int(hr.shape[0]/TRIAL*hr.shape[1])))
hr= hr[num_trials_to_remove:]

pupil = np.array(pupil_data)
pupil = pupil.reshape((TRIAL, int(pupil.shape[0]/TRIAL*pupil.shape[1])))
pupil = pupil[num_trials_to_remove:]

eda = np.array(eda_data)
eda = eda.reshape((TRIAL,int(eda.shape[0]/TRIAL*eda.shape[1])))
eda = eda[num_trials_to_remove:]
'''different types of observable data

1) gaze
2) fisio
    2.1) heart rate variability
    2.2) eda phasic value
3) social anxiety
4) aspettativa del dolore
'''
'''print(pupil.shape)
print(hr.shape)
print(eda.shape)'''
N_pupil = pupil.shape[0]
D_pupil = pupil.shape[1]

N_hr = hr.shape[0]
D_hr = hr.shape[1]

N_eda = eda.shape[0]
D_eda = eda.shape[1]
K = latent_space

print(K)
print(N_pupil,D_pupil)
print(N_hr,D_hr)
print(N_eda,D_eda)
print(N_e, D_e)
print(N_sub, D_sub)

In [ ]:
# model
with pm.Model() as sPPCA:
    #dati osservabili
    hr_data = pm.MutableData("hr_data", hr.T)
    #pupil_data = pm.MutableData("pupil_data", pupil.T)
    #eda_data = pm.MutableData("eda_data", eda.T)
    e_data = pm.ConstantData("e_data", global_e_labels.T)
    #aggiunta
    subject_idx = pm.Data("subject_idx", global_subject)

    #matrici pesi
    Whr = pm.Normal('Whr', mu=at.zeros([D_hr, K]), sigma=2.0 * at.ones([D_hr, K]), shape=[D_hr, K])
    #Wpupil = pm.Normal('Wpupil', mu=at.zeros([D_pupil, K]), sigma=2.0 * at.ones([D_pupil, K]), shape=[D_pupil, K])

    #Weda = pm.Normal('Weda', mu=at.zeros([D_eda, K]), sigma=2.0 * at.ones([D_eda, K]), shape=[D_eda, K])

    #weight matrix for pain expectation.
    #check mu,sigma,shape
    We = pm.Normal('W_e', mu=at.zeros([D_e, K]), sigma=2.0 * at.ones([D_e,K]), shape=[D_e, K])

    #latent space
    c = pm.Normal('c', mu=at.zeros([N_hr,K]), sigma=at.ones([N_hr,K]), shape=[N_hr,K])

    # dati dell'hrv interpretati come una gaussiana
    '''mu_hr = pm.Normal('mu_hr', Whr.dot(c.T), at.ones([D_hr,N_hr])) # hyperprior 1
    sigma_hr = pm.Exponential('sigma_hr', at.ones([D_hr,N_hr]))# hyperprior 2'''

    a_hr = pm.Normal('mu_hr', Whr.dot(c.T), 10) # hyperprior 1
    sigma_hr = pm.Exponential('sigma_hr', 1)# hyperprior 2
    a_subjects= pm.Normal("a_subjects",mu=a_hr, sigma=sigma_hr) #dims=[D_hr,N_hr]
    theta = a_subjects[subject_idx]
    sigma2_hr = pm.Exponential("sigma2_hr",1.0)

    x_hr = pm.Normal('x_hr', mu=theta, sigma=sigma2_hr ,shape=[D_hr, N_hr], observed=hr_data)

    '''# dati della dilatazione pupille interpretati come una gaussiana
    mu_pupil = pm.Normal('mu_pupil', Wpupil.dot(c.T), at.ones([D_pupil,N_pupil])) # hyperprior 1
    sigma_pupil = pm.Exponential('sigma_pupil', at.ones([D_pupil,N_pupil]))# hyperprior 2
    x_pupil = pm.Normal('x_pupil', mu=mu_pupil, sigma=sigma_pupil, shape=[D_pupil, N_pupil], observed=pupil_data)

    #eda
    mu_eda = pm.Normal('mu_eda', Weda.dot(c.T), at.ones([D_eda,N_eda])) # hyperprior 1
    sigma_eda = pm.Exponential('sigma_eda', at.ones([D_eda,N_eda]))# hyperprior 2
    x_eda = pm.Normal('x_eda', mu=mu_eda, sigma=sigma_eda, shape=[D_eda,N_eda], observed=eda_data)'''

    # pain expectation. ciò che dovremmo inferire dato c
    # due strade: binary o multiclass (1-4)
    # p = probability of success?
    x_e = pm.Bernoulli('x_e' , p=pm.math.sigmoid(We.dot(c.T)) , shape =[D_e, N_e], observed=e_data)

    #x_hr = pm.Bernoulli('x_hr', p=pm.math.sigmoid(Whr.dot(c.T)), shape=[D_hr, N_hr], observed=hr_data)
    #x_eda = pm.Bernoulli('x_eda', p=pm.math.sigmoid(Weda.dot(c.T)), shape=[D_eda, N_eda], observed=eda_data)

In [ ]:
gv = pm.model_to_graphviz(sPPCA)
gv.view()

In [ ]:
with sPPCA:
        approx = pm.fit(100000, callbacks=[pm.callbacks.CheckParametersConvergence(tolerance=1e-4)], progressbar=True)
        trace = approx.sample(500)


In [ ]:
with sPPCA:
    # update values of predictors:
    # use the updated values and predict outcomes and probabilities:
    posterior_predictive = pm.sample_posterior_predictive(
        trace, random_seed=123, progressbar=False)
e_pred = posterior_predictive.posterior_predictive["x_e"]
e_pred_mode = np.squeeze(stats.mode(e_pred[0], keepdims=False)[0])[:,np.newaxis]

train_accuracy_exp = accuracy_score(global_e_labels, e_pred_mode)